<a href="https://colab.research.google.com/github/cuilongjin/static/blob/main/colab-script/GoogleDrive%E7%A6%BB%E7%BA%BF%E4%B8%8B%E8%BD%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Google Drive 离线下载


###使用说明

**挂载团队盘的方式如下**

默认挂载个人盘，如果容量不够大或者有需要可以修改为团队盘，提供两种方法：

第一种方法 创建团队盘的快捷方式，并移动到个人盘，在第三步中修改为快捷方式的路径，即可下载到团队盘。

第二种方法 将第三步中的路径，修改为`/content/drive/Shareddrives/你的团队盘名称/文件夹` 。不要下载到团队盘根目录，要在团队盘下建立一个文件夹进行下载。

**关于文件下载大小的说明**

个人对此的建议是不要超过70G因为你的实例有100G，但是实际可用其实只有70G。所以每次下载无论你是单文件，还是多文件都别超过70G（具体看右上角的容量），分批下载。不要着急。不然被封24小时了。

**感谢：**

https://fklc.dev/ 加拿大大佬提供google colab下载技术

https://lemongeek.com/712.html 提供的一些说明

### ①安装 libtorrent 并初始化

In [ ]:
#@title 点击这里运行，运行完成后进行下一步
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

### ②挂载 Google Drive

传输文件需要挂载 Google Drive.

In [ ]:
#@title 点击这里运行，运行后会提示打开链接，登录你的账户，获取授权码，并在输入框中输入。完成后进行下一步
from google.colab import drive

drive.mount("/content/drive")

### ③添加 Torrent 种子文件

上传种子文件或添加磁力链接，磁力链接请往下看。如需修改下载路径，在运行前修改，见下方注释。点击后上传种子文件。

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/离线下载", # 此处修改下载路径
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### OR 添加磁力链接

添加磁力链接。如需修改下载路径，在运行前修改，见下方注释。点击后粘贴链接，回车键输入，完成后输入exit退出。

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### ④开始下载

Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
#@title 点击开始下载。可以关闭此页面进行。
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)